<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-06-04 14:13:17
-------------------
qualified stocks: 93
with latest results: 32
still star stocks: 18


-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  1.03 L
Current:  1.32 C
-------------------
Today PnL: 49.54 K (0.38%)
Current PnL: -16.13 L (-11.24%)
CY Booked + Current PnL: -11.66 L (-8.13%)
-------------------
Total profit:  4.47 L
Total loss:  -20.60 L
-------------------
Total Booked + Current PnL: 15.32 L (13.14%)
Total Booked PnL: 31.45 L (26.99%)
Curr Year Booked PnL: 4.47 L (3.39%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.11 C
Est FTT PnL: 79.24 L (60.04%)
-------------------
Est LTT:  2.73 C
Est LTT PnL: 1.41 C (107.18%)
Deployed:  1.17 C
Current:  1.32 C
CAGR/XIRR %: 8.86%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,PIDILITIND,2504.06,3202.00,-16.12,X-LC,38.71,92937.0,17815.0,3123.0,0.06,23.72,3.36,27.87,9.0,5.70,0.71,16.55,X40,NTT,CHEMICALS
28,GILLETTE,8109.44,10330.69,14.52,H-SC,94.62,168827.0,30967.0,6787.0,2.94,22.46,4.02,27.39,116.0,4.56,1.29,41.26,X40,ATH,FMCG
49,LICI,785.05,983.00,-3.65,H-LC,88.17,236555.0,41863.0,7239.0,0.02,21.50,3.06,25.21,31.0,5.78,1.81,30.81,XY25,NTT,INSURANCE
4,ADANIPORTS,1103.69,1583.00,1.78,M-LC,59.14,102148.0,23786.0,10245.0,0.45,30.35,10.03,43.43,70.0,2.32,0.78,36.62,XY24,NTT,MISC
85,UNIONBANK,123.87,163.00,-0.31,M-LC,91.40,174541.0,33701.0,10787.0,1.23,23.93,6.18,31.59,79.0,3.12,1.33,51.61,XY24,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TITAGARH,1097.23,1548.00,-6.68,H-SC,96.77,163782.0,-28233.0,107113.0,4.13,-14.70,65.40,41.08,128.0,-0.26,1.25,38.62,XY24,NTT,ENGINEERING
59,RAJESHEXPO,517.65,583.00,1881.06,M-SC,56.99,54065.0,-83112.0,100431.0,3.32,-60.59,185.76,12.62,209.0,-0.83,0.41,34.46,X40N,NTT,JEWELLERY
28,GILLETTE,8109.44,10330.69,14.52,H-SC,94.62,168827.0,30967.0,6787.0,2.94,22.46,4.02,27.39,116.0,4.56,1.29,41.26,X40,ATH,FMCG
29,GREENPANEL,375.16,537.00,186.82,M-SC,74.19,127929.0,-55149.0,134121.0,2.34,-30.12,104.84,43.14,191.0,-0.41,0.98,20.34,XY24,NTT,MISC
1,AARTIIND,487.04,919.00,39.94,M-SC,58.06,80277.0,-1059.0,73197.0,2.15,-1.30,91.18,88.69,205.0,-0.01,0.61,35.37,XR,NTT,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SATIN,199.77,274.00,-22.47,H-SC,7.53,152153.0,-43821.0,116640.0,-2.65,-22.36,76.66,37.16,122.0,-0.38,1.16,13.63,XY24,NTT,FINANCE
52,MASFIN,326.60,399.50,-24.12,H-SC,62.37,86370.0,-11610.0,33477.0,-1.39,-11.85,38.76,22.32,127.0,-0.35,0.66,25.27,XR,ATH,FINANCE
88,VBL,492.64,672.85,-17.10,H-LC,2.15,251757.0,-13283.0,110244.0,-1.36,-5.01,43.79,36.58,26.0,-0.12,1.92,7.52,AR,ATH,FMCG
53,MEDANTA,1022.60,1486.00,3.83,X-MC,23.66,182917.0,27482.0,42949.0,-1.13,17.68,23.48,45.32,66.0,0.64,1.40,24.46,XY24,NTT,HEALTHCARE
70,SHALBY,261.39,327.00,837.80,M-SC,20.43,127512.0,-53631.0,99102.0,-1.08,-29.61,77.72,25.10,206.0,-0.54,0.97,2.06,XY24,NTT,HEALTHCARE


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1461.46,2275.00,-13.14,X-LC,49.46,278059.0,16458.0,129158.0,0.67,6.29,46.45,55.67,1.0,0.13,2.12,14.32,X40,BTT,IT
80,TCS,3794.03,4998.00,-19.05,X-LC,18.28,263679.0,-32255.0,126170.0,0.16,-10.90,47.85,31.73,2.0,-0.26,2.01,5.54,X40,BTT,IT
50,LTIM,5564.16,7197.33,-3.66,X-LC,78.49,185562.0,-14748.0,73538.0,1.85,-7.36,39.63,29.35,3.0,-0.20,1.42,28.20,X200,ATH,IT
6,ASIANPAINT,2961.56,4250.00,-22.87,X-LC,4.30,153340.0,-48046.0,135660.0,-0.03,-23.86,88.47,43.51,5.0,-0.35,1.17,5.76,X40,BTT,PAINTS
34,HINDUNILVR,2413.81,2723.00,-16.14,X-LC,25.81,263502.0,-6845.0,41475.0,0.02,-2.53,15.74,12.81,6.0,-0.17,2.01,9.06,XY25,NTT,FMCG


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,14.35,H-SC,98.92,132250.0,34604.0,13066.0,0.99,35.44,9.88,48.82,134.0,2.65,1.01,46.92,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,14.52,H-SC,94.62,168827.0,30967.0,6787.0,2.94,22.46,4.02,27.39,116.0,4.56,1.29,41.26,X40,ATH,FMCG
85,UNIONBANK,123.87,163.00,-0.31,M-LC,91.40,174541.0,33701.0,10787.0,1.23,23.93,6.18,31.59,79.0,3.12,1.33,51.61,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.00,1.78,M-LC,59.14,102148.0,23786.0,10245.0,0.45,30.35,10.03,43.43,70.0,2.32,0.78,36.62,XY24,NTT,MISC
49,LICI,785.05,983.00,-3.65,H-LC,88.17,236555.0,41863.0,7239.0,0.02,21.50,3.06,25.21,31.0,5.78,1.81,30.81,XY25,NTT,INSURANCE


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,99.34,M-MC,83.87,226960.0,1998.0,165658.0,1.22,0.89,72.99,74.53,170.0,0.01,1.73,31.77,XY24,BTT,STEEL
51,LUXIND,1448.19,2442.00,39.47,M-SC,68.82,144714.0,7136.0,87277.0,1.38,5.19,60.31,68.62,183.0,0.08,1.11,25.46,X40N,NTT,TEXTILES
28,GILLETTE,8109.44,10330.69,14.52,H-SC,94.62,168827.0,30967.0,6787.0,2.94,22.46,4.02,27.39,116.0,4.56,1.29,41.26,X40,ATH,FMCG
55,PFIZER,4245.50,6318.30,14.35,H-SC,98.92,132250.0,34604.0,13066.0,0.99,35.44,9.88,48.82,134.0,2.65,1.01,46.92,X40,ATH,PHARMA
57,POLYCAB,5005.95,7541.35,5.53,H-MC,34.41,100249.0,15148.0,27949.0,0.44,17.80,27.88,50.65,90.0,0.54,0.77,26.02,X40N,ATH,CABLES


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,4.44,H-SC,67.74,117323.0,15319.0,29706.0,-0.10,15.02,25.32,44.14,112.0,0.52,0.90,33.70,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-27.01,M-MC,60.22,142387.0,13235.0,73828.0,0.43,10.25,51.85,67.42,156.0,0.18,1.09,38.48,XR,NTT,BANKS
38,INDIAMART,2327.09,4911.36,-51.65,H-SC,55.91,126071.0,2735.0,134228.0,0.72,2.22,106.47,111.05,108.0,0.02,0.96,23.53,AR,ATH,MISC
92,WIPRO,243.46,420.00,-10.62,M-LC,32.26,153351.0,2406.0,107054.0,0.46,1.59,69.81,72.51,68.0,0.02,1.17,15.14,XR,NTT,IT
1,AARTIIND,487.04,919.00,39.94,M-SC,58.06,80277.0,-1059.0,73197.0,2.15,-1.30,91.18,88.69,205.0,-0.01,0.61,35.37,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,MASFIN,326.60,399.5,-24.12,H-SC,62.37,86370.0,-11610.0,33477.0,-1.39,-11.85,38.76,22.32,127.0,-0.35,0.66,25.27,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,1.78,M-LC,59.14,102148.0,23786.0,10245.0,0.45,30.35,10.03,43.43,70.0,2.32,0.78,36.62,XY24,NTT,MISC
84,UJJIVANSFB,52.77,60.0,35.36,M-SC,51.61,117288.0,-25191.0,44710.0,-0.16,-17.68,38.12,13.70,221.0,-0.56,0.90,38.43,X40N,NTT,BANKS
87,VALIANTORG,512.64,838.0,-378.23,H-SC,100.00,118313.0,-14973.0,99572.0,-0.97,-11.23,84.16,63.47,130.0,-0.15,0.90,98.32,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.3,14.35,H-SC,98.92,132250.0,34604.0,13066.0,0.99,35.44,9.88,48.82,134.0,2.65,1.01,46.92,X40,ATH,PHARMA


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.0,-378.23,H-SC,100.00,118313.0,-14973.0,99572.0,-0.97,-11.23,84.16,63.47,130.0,-0.15,0.90,98.32,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.3,14.35,H-SC,98.92,132250.0,34604.0,13066.0,0.99,35.44,9.88,48.82,134.0,2.65,1.01,46.92,X40,ATH,PHARMA
7,ASIANTILES,75.41,137.0,6522.22,M-SC,97.85,72650.0,-21160.0,97780.0,0.74,-22.56,134.59,81.67,214.0,-0.22,0.55,40.69,XR,NTT,CERAMICS
81,TITAGARH,1097.23,1548.0,-6.68,H-SC,96.77,163782.0,-28233.0,107113.0,4.13,-14.70,65.40,41.08,128.0,-0.26,1.25,38.62,XY24,NTT,ENGINEERING
74,STARHEALTH,551.56,761.0,4.17,M-SC,95.70,170410.0,-29806.0,105825.0,-0.66,-14.89,62.10,37.97,195.0,-0.28,1.30,36.83,XY24,NTT,INSURANCE


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.57
1,25,41.88
2,50,70.48


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    31.49
XY25    14.53
X40     14.44
AR      13.37
X40N    13.24
XR      10.35
X200     1.42
SR       1.12
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.24
X-LC    18.63
M-SC    17.38
H-MC     8.48
M-LC     7.67
H-LC     7.56
X-MC     5.53
M-MC     5.52
X-SC     2.74
L-SC     2.28
L-LC     1.11
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.67,-0.36,31.51
IT,10.14,-17.55,81.39
BANKS,9.70,-10.53,61.34
MISC,6.40,-29.12,86.24
FINANCE,5.81,-25.57,85.20
INSURANCE,5.28,3.51,25.40
PAINTS,5.27,-19.24,54.52
ELECTRICAL,5.10,-5.18,50.80
AUTO,4.29,-11.11,59.65


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2848770.0,26
AR,1230331.0,12
X40N,1215121.0,17
XR,1094671.0,14
X40,787309.0,11
XY25,475194.0,10
SR,198683.0,2
X200,73538.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2237280.0,21
M-SC,2037941.0,21
X-LC,866300.0,12
M-MC,546125.0,5
H-MC,516882.0,8
X-SC,349407.0,4
X-MC,308973.0,5
M-LC,300261.0,6
L-SC,298427.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      934646.0      6
M-SC       XY24      752568.0      6
H-SC       AR        545407.0      3
           X40N      498667.0      6
X-LC       X40       482605.0      5
M-SC       X40N      471547.0      6
M-MC       XY24      390396.0      3
M-SC       XR        290115.0      4
H-SC       XR        238707.0      4
M-SC       X40       222349.0      2
H-LC       AR        210782.0      3
M-LC       XY24      178726.0      4
X-LC       AR        173352.0      2
H-MC       XY24      173283.0      2
L-SC       XR        156583.0      2
X-MC       XY24      138347.0      2
X-LC       XY25      136805.0      4
M-SC       XY25      121389.0      1
X-SC       X40N      120889.0      2
           XR        115634.0      1
           SR        112884.0      1
H-MC       XR        112750.0      1
M-LC       XR        107054.0      1
X-MC       X40N       96069.0      2
M-SC       AR         94174.0      1
           SR         85799.0      1
M-MC       AR         81901.0      1
L-SC       XY24       80354.0      1
H-MC       XY25       77173.0      1
X-MC       XY25       74557.0      1
M-MC       XR         73828.0      1
X-LC       X200       73538.0      1
H-MC       AR         63225.0      1
           X40        62502.0      2
L-SC       AR         61490.0      1
L-LC       XY25       43550.0      1
H-MC       X40N       27949.0      1
H-SC       X40        19853.0      2
M-LC       XY25       14481.0      1
H-LC       XY25        7239.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 59.0 seconds
